<a href="https://colab.research.google.com/github/Nazarique/meta-heur-sticas-bioinspiradas/blob/main/Lista_3/Lista_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pyswarms.readthedocs.io/en/latest/_modules/pyswarms/discrete/binary.html
https://github.com/sraaphorst/complement_binary_particle_swarm_optimization/blob/master/cbpso.py

o problema pra resolver eh esse
https://tracer.lcc.uma.es/problems/onemax/onemax.html



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/8° Período/Meta/lista/lista 3

/content/drive/My Drive/8° Período/Meta/lista/lista 3


In [ ]:
! git clone git://github.com/ljvmiranda921/pyswarms

Cloning into 'pyswarms'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 3003 (delta 11), reused 6 (delta 0), pack-reused 2975
Receiving objects: 100% (3003/3003), 54.93 MiB | 16.90 MiB/s, done.
Resolving deltas: 100% (1814/1814), done.


In [ ]:
! git pull

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [ ]:
! pip install pyswarms

In [ ]:
import pyswarms

# link1

# -*- coding: utf-8 -*-

r"""
A Binary Particle Swarm Optimization (binary PSO) algorithm.

It takes a set of candidate solutions, and tries to find the best
solution using a position-velocity update method. Unlike
:mod:`pyswarms.single.gb` and :mod:`pyswarms.single.lb`, this technique
is often applied to discrete binary problems such as job-shop scheduling,
sequencing, and the like.

The update rule for the velocity is still similar, as shown in the
proceeding equation:

.. math::

   v_{ij}(t + 1) = w * v_{ij}(t) + c_{1}r_{1j}(t)[y_{ij}(t) − x_{ij}(t)] + c_{2}r_{2j}(t)[\hat{y}_{j}(t) − x_{ij}(t)]

For the velocity update rule, a particle compares its current position
with respect to its neighbours. The nearest neighbours are being
determined by a kD-tree given a distance metric, similar to local-best
PSO. The neighbours are computed for every iteration. However, this whole
behavior can be modified into a global-best PSO by changing the nearest
neighbours equal to the number of particles in the swarm. In this case,
all particles see each other, and thus a global best particle can be established.

In addition, one notable change for binary PSO is that the position
update rule is now decided upon by the following case expression:

.. math::

   X_{ij}(t+1) = \left\{\begin{array}{lr}
        0, & \text{if } \text{rand() } \geq S(v_{ij}(t+1))\\
        1, & \text{if } \text{rand() } < S(v_{ij}(t+1))
        \end{array}\right\}

Where the function :math:`S(x)` is the sigmoid function defined as:

.. math::

   S(x) = \dfrac{1}{1 + e^{-x}}

This enables the algorithm to output binary positions rather than
a stream of continuous values as seen in global-best or local-best PSO.

This algorithm was adapted from the standard Binary PSO work of J. Kennedy and
R.C. Eberhart in Particle Swarm Optimization [SMC1997]_.

.. [SMC1997] J. Kennedy and R.C. Eberhart, "A discrete binary version of
    particle swarm algorithm," Proceedings of the IEEE International
    Conference on Systems, Man, and Cybernetics, 1997.
"""

In [ ]:

# Import standard library
import logging

# Import modules
import numpy as np
import multiprocessing as mp

from collections import deque

from pyswarms.backend.operators import compute_pbest, compute_objective_function
from pyswarms.backend.topology import Ring
from pyswarms.backend.handlers import BoundaryHandler, VelocityHandler
from pyswarms.base import DiscreteSwarmOptimizer
from pyswarms.utils.reporter import Reporter


class BinaryPSO(DiscreteSwarmOptimizer):
    def __init__(
        self,
        n_particles,
        dimensions,
        options,
        init_pos=None,
        velocity_clamp=None,
        vh_strategy="unmodified",
        ftol=-np.inf,
        ftol_iter=1,
    ):
        """Initialize the swarm

        Attributes
        ----------
        n_particles : int
            number of particles in the swarm.
        dimensions : int
            number of dimensions in the space.
        options : dict with keys :code:`{'c1', 'c2', 'w', 'k', 'p'}`
            a dictionary containing the parameters for the specific
            optimization technique
                * c1 : float
                    cognitive parameter
                * c2 : float
                    social parameter
                * w : float
                    inertia parameter
                * k : int
                    number of neighbors to be considered. Must be a
                    positive integer less than :code:`n_particles`
                * p: int {1,2}
                    the Minkowski p-norm to use. 1 is the
                    sum-of-absolute values (or L1 distance) while 2 is
                    the Euclidean (or L2) distance.
        init_pos : numpy.ndarray, optional
            option to explicitly set the particles' initial positions. Set to
            :code:`None` if you wish to generate the particles randomly.
        velocity_clamp : tuple, optional
            a tuple of size 2 where the first entry is the minimum velocity
            and the second entry is the maximum velocity. It
            sets the limits for velocity clamping.
        vh_strategy : String
            a strategy for the handling of the velocity of out-of-bounds particles.
            Only the "unmodified" and the "adjust" strategies are allowed.
        ftol : float
            relative error in objective_func(best_pos) acceptable for
            convergence
        ftol_iter : int
            number of iterations over which the relative error in
            objective_func(best_pos) is acceptable for convergence.
            Default is :code:`1`
        """
        # Initialize logger
        self.rep = Reporter(logger=logging.getLogger(__name__))
        # Assign k-neighbors and p-value as attributes
        self.k, self.p = options["k"], options["p"]
        self.ftol_iter = ftol_iter
        # Initialize parent class
        super(BinaryPSO, self).__init__(
            n_particles=n_particles,
            dimensions=dimensions,
            binary=True,
            options=options,
            init_pos=init_pos,
            velocity_clamp=velocity_clamp,
            ftol=ftol,
            #ftol_iter=ftol_iter
        )
        # Initialize the resettable attributes
        self.reset()
        # Initialize the topology
        self.top = Ring(static=False)
        self.vh = VelocityHandler(strategy=vh_strategy)
        self.name = __name__


    def optimize(
        self, objective_func, iters, n_processes=None, verbose=True, **kwargs
    ):
        """Optimize the swarm for a number of iterations

        Performs the optimization to evaluate the objective
        function :code:`f` for a number of iterations :code:`iter.`

        Parameters
        ----------
        objective_func : function
            objective function to be evaluated
        iters : int
            number of iterations
        n_processes : int, optional
            number of processes to use for parallel particle evaluation
            Defaut is None with no parallelization.
        verbose : bool
            enable or disable the logs and progress bar (default: True = enable logs)
        kwargs : dict
            arguments for objective function

        Returns
        -------
        tuple
            the local best cost and the local best position among the
            swarm.
        """
        # Apply verbosity
        if verbose:
            log_level = logging.INFO
        else:
            log_level = logging.NOTSET

        self.rep.log("Obj. func. args: {}".format(kwargs), lvl=logging.DEBUG)
        self.rep.log(
            "Optimize for {} iters with {}".format(iters, self.options),
            lvl=log_level,
        )
        # Populate memory of the handlers
        self.vh.memory = self.swarm.position

        # Setup Pool of processes for parallel evaluation
        pool = None if n_processes is None else mp.Pool(n_processes)

        self.swarm.pbest_cost = np.full(self.swarm_size[0], np.inf)
        ftol_history = deque(maxlen=self.ftol_iter)
        for i in self.rep.pbar(iters, self.name) if verbose else range(iters):
            # Compute cost for current position and personal best
            self.swarm.current_cost = compute_objective_function(
                self.swarm, objective_func, pool, **kwargs
            )
            self.swarm.pbest_pos, self.swarm.pbest_cost = compute_pbest(
                self.swarm
            )
            best_cost_yet_found = np.min(self.swarm.best_cost)
            # Update gbest from neighborhood
            self.swarm.best_pos, self.swarm.best_cost = self.top.compute_gbest(
                self.swarm, p=self.p, k=self.k
            )
            if verbose:
                # Print to console
                self.rep.hook(best_cost=self.swarm.best_cost)
            # Save to history
            hist = self.ToHistory(
                best_cost=self.swarm.best_cost,
                mean_pbest_cost=np.mean(self.swarm.pbest_cost),
                mean_neighbor_cost=np.mean(self.swarm.best_cost),
                position=self.swarm.position,
                velocity=self.swarm.velocity,
            )
            self._populate_history(hist)
            # Verify stop criteria based on the relative acceptable cost ftol
            relative_measure = self.ftol * (1 + np.abs(best_cost_yet_found))
            delta = (
                np.abs(self.swarm.best_cost - best_cost_yet_found)
                < relative_measure
            )
            if i < self.ftol_iter:
                ftol_history.append(delta)
            else:
                ftol_history.append(delta)
                if all(ftol_history):
                    break
            # Perform position velocity update
            self.swarm.velocity = self.top.compute_velocity(
                self.swarm, self.velocity_clamp, self.vh
            )
            self.swarm.position = self._compute_position(self.swarm)
        # Obtain the final best_cost and the final best_position
        final_best_cost = self.swarm.best_cost.copy()
        final_best_pos = self.swarm.pbest_pos[
            self.swarm.pbest_cost.argmin()
        ].copy()
        self.rep.log(
            "Optimization finished | best cost: {}, best pos: {}".format(
                final_best_cost, final_best_pos
            ),
            lvl=log_level,
        )
        # Close Pool of Processes
        if n_processes is not None:
            pool.close()

        return (final_best_cost, final_best_pos)


    def _compute_position(self, swarm):
        """Update the position matrix of the swarm

        This computes the next position in a binary swarm. It compares the
        sigmoid output of the velocity-matrix and compares it with a randomly
        generated matrix.

        Parameters
        ----------
        swarm: pyswarms.backend.swarms.Swarm
            a Swarm class
        """
        return (
            np.random.random_sample(size=swarm.dimensions)
            < self._sigmoid(swarm.velocity)
        ) * 1

    def _sigmoid(self, x):
        """Helper method for the sigmoid function

        Parameters
        ----------
        x : numpy.ndarray
            Input vector for sigmoid computation

        Returns
        -------
        numpy.ndarray
            Output sigmoid computation
        """
        return 1 / (1 + np.exp(-x))
2

2

In [ ]:
def x(x):
  return x**2 

In [ ]:
n_particles, dimensions, options = 10, 2, {'c1': [1.5, 1.5], 'c2': [1.5, 1.5], 'w':1.5, 'k':1, 'p':1}
problema = BinaryPSO(n_particles, dimensions, options)

In [ ]:
problema.optimize(objective_func=x, iters=5)

2020-10-20 18:42:56,980 - __main__ - INFO - Optimize for 5 iters with {'c1': [1.5, 1.5], 'c2': [1.5, 1.5], 'w': 1.5, 'k': 1, 'p': 1}











__main__:   0%|          |0/5

ValueError: ignored

# link2


In [ ]:
#!/usr/bin/env python3
#
# By Sebastian Raaphorst, 2019.
#
# Complimentary binary particle swarm optimization algorithm, as per:
# Li-Yeh Chuang, Chih-Jen Hsiao, Cheng-Hong Yang, An Improved Binary Particle Swarm Optimization with Complementary
# Distribution Strategy for Feature Selection, 2009 International Conference on Machine Learning and Computing.
# https://pdfs.semanticscholar.org/7b5e/7a8b7f86366e446ecd37121e5cd20199d329.pdf

import numpy as np
from math import exp


class CBPSO:
    def __init__(self,
                 dimensions,
                 fitness,
                 stop_func=lambda _: False,
                 num_particles=50,
                 rounds=100,
                 max_rounds_to_complement=3,
                 w=1,
                 c1=1,
                 c2=1,
                 vmax=1,
                 end_round=None):
        self._dimensions = dimensions
        self._fitness = fitness
        self._stop_func = stop_func
        self._num_particles = num_particles
        self._particles = None
        self._rounds = rounds
        self._max_rounds_to_complement = max_rounds_to_complement
        self._w = w
        self._c1 = c1
        self._c2 = c2
        self._vmax = vmax
        self._end_round = end_round

    @staticmethod
    def _S(x):
        return 1.0 / (1.0 + exp(-x))

    def run(self):
        # Shorthand to avoid self._ references:
        p, d = self._num_particles, self._dimensions

        # Initialize all the particle positions and record as the best positions.
        self._particles = np.random.randint(0, 2, (p, d))
        best_particles = np.zeros((p, d))
        best_fitnesses = np.zeros(p)

        # Structures to keep track of the global best particle.
        best_particle = None
        best_particle_fitness = None

        # Randomize the velocity dimensions to be between [-vmax, vmax].
        velocities = (np.random.random((p, d)) * 2 - 1) * self._vmax

        # Keep track of the number of rounds where the best_particle has not changed.
        num_rounds = 0

        for t in range(self._rounds):
            if best_particle is not None:
                print("Round {}: {}".format(t, best_particle_fitness))
            # Calculate the fitness of the particles and update the best fitnesses and best_particles if we
            # have improved.
            fitnesses = np.apply_along_axis(self._fitness, 1, self._particles)
            fitness_comparison = fitnesses > best_fitnesses
            best_particles = np.where(fitness_comparison.reshape(p, 1), self._particles, best_particles)
            best_fitnesses = np.where(fitness_comparison, fitnesses, best_fitnesses)

            # Keep track of the best particle.
            best_particle_candidate = np.argmax(best_fitnesses)
            best_particle_candidate_fitness = best_fitnesses[best_particle_candidate]
            if best_particle is None or best_particle_fitness < best_particle_candidate_fitness:
                best_particle, best_particle_fitness = best_particle_candidate, best_particle_candidate_fitness
                num_rounds = 0
            else:
                num_rounds += 1

            # If we have not changed, take the complement of a random half of the vectors.
            # There is probably a numpy guru way to do this without iterating, but I'm not sure what it is.
            # With fancy indexing, we can do this in one step.
            if num_rounds >= self._max_rounds_to_complement:
                rows = np.random.choice(p, p // 2)
                self._particles[[rows], :] = 1 - self._particles[[rows], :]
                num_rounds = 0

            # Update the particle velocities.
            velocities = self._w * velocities +\
                         self._c1 * np.random.random((p, d)) * (np.tile(best_fitnesses, d).reshape((p, d))
                                                                - np.tile(fitnesses, d).reshape(p, d)) +\
                         self._c2 * np.random.random((p, d)) * (np.tile(np.tile(best_particle, p), d).reshape(p, d)
                                                                - np.tile(fitnesses, d).reshape(p, d))

            # Clamp them to being in [-vmax, vmax].
            velocities = np.where(np.where(velocities < -self._vmax, -self._vmax, velocities) > self._vmax,
                                  self._vmax, velocities)

            # Now calculate the new positions.
            self._particles = np.where(np.random.random((p, d)) < np.vectorize(CBPSO._S)(velocities), 1, 0)

            # Check the termination condition.
            if best_particle is not None and self._stop_func(self._particles[best_particle]):
                return True, self._particles[best_particle], best_particle_fitness

        # Failure.
        return False, self._particles[best_particle], best_particle_fitness


# link3


In [ ]:
import numpy as np

def initialize(num_inds, num_params, domain=[0, 2], real_valued=False):
    lower_bound = domain[0]
    upper_bound = domain[1]
    if real_valued:
        pop = np.random.uniform(low=lower_bound, 
                                high=upper_bound, 
                                size=(num_inds, num_params))
    else:
        pop = np.random.randint(low=lower_bound, 
                                high=upper_bound+1, 
                                size=(num_inds, num_params))
    return pop

def evaluate(pop, func):
    f_pop = np.array(list(map(func, pop)))
    return f_pop

def max_gen_reached(gen, max_gen):
    return gen == max_gen

def pop_converge(pop):
    return len(np.unique(pop, axis=0)) == 1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

def scatter_plot(ax_lim, ax, raw_data, title='', hold=True):
    x, y = raw_data[:, 0], raw_data[:, 1]
    ax.plot(x, y, "g.", label='genome')
    # ax.suptitle(title, size=15)
    # ax.title(title, loc='center')
    (xlim, ylim) = ax_lim
    ax.legend(loc=1)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    if not hold:
        plt.show()

def contour_plot(ax, coords, f_dict, title='', hold=True):
    (X, Y, Z) = coords
    global_minimums = f_dict['global minimum']
    global_maximums = f_dict['global maximum']
    if type(global_minimums) != type(None):
        global_minimums = np.reshape(global_minimums, (-1, 2))
        ax.plot(global_minimums[:, 0], global_minimums[:, 1], 'rx', label='global minimum', markersize=10)
    if type(global_maximums) != type(None):
        global_maximums = np.reshape(global_maximums, (-1, 2))
        ax.plot(global_maximums[:, 0], global_maximums[:, 1], 'bp', label='global maximum', markersize=10)

    ax.contour(X, Y, Z, 25, cmap=cm.seismic)

    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    ax.set_title(f_dict['name'])

    # ax.legend(loc=1)
    if not hold:
        plt.show()

def scatter_3D(ax_lim, ax, X, Y, title='', hold=True):
    x = X[:, 0]
    y = X[:, 1]
    z = Y
    ax.scatter3D(x, y, z, label='Genome')
    (x_lim, y_lim, z_lim) = ax_lim
    ax.set_xlim(x_lim)
    ax.set_ylim(y_lim)
    ax.set_zlim(z_lim)
    ax.legend()
    if not hold:
        plt.show()


def contour_3D(ax, coords, f_dict, title='', hold=True):
    (X, Y, Z) = coords

    ax.contour3D(X, Y, Z, 20, cmap=cm.rainbow)
    
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("F(x, y)")
    ax.set_title(f_dict['name'])

    # fig.colorbar(surf, shrink=0.5, aspect=10)
    # fig.suptitle(title)
        
    if not hold:
        plt.show()

def get_plot_data(f_dict):
    step = 1 if not f_dict['real valued'] else 0.25
    (lower_bound, upper_bound) = f_dict['D']
    x_axis = np.arange(lower_bound, upper_bound, step)
    y_axis = np.arange(lower_bound, upper_bound, step)
    n = len(x_axis)
    if n % 2 != 0:
        x_axis = x_axis[:-1]
        y_axis = y_axis[:-1]
    x_mesh, y_mesh = np.meshgrid(x_axis, y_axis)

    if f_dict['multi dims']:
        # xy1, xy2 = x_mesh.copy(), y_mesh.copy()
        # num_half = len(xy1) // 2
        # xy1[:, :num_half], xy2[:, num_half:] = xy2[:, num_half:], xy1[:, :num_half].copy()

        # z1 = np.array(list(map(f_dict['function'], xy1)))[:, np.newaxis]
        # z2 = np.array(list(map(f_dict['function'], xy2)))[:, np.newaxis]
        # z_mesh = np.dstack((z1, z2))
        n = len(x_mesh)
        z_mesh = [f_dict['function'](np.array([x_mesh[i], y_mesh[i]])) for i in range(n)]
        z_mesh = np.array(z_mesh)
    else:
        z_mesh = np.array(f_dict['function']([x_mesh, y_mesh]))

    
    return (x_mesh, y_mesh, z_mesh)

# dict = f_dict.onemax_dict
# dict = f_dict.himmelblau_dict
# # dict = f_dict.cross_in_tray_dict
# dict = f_dict.rastrigin_dict
# coords = get_plot_data(dict)
# print("i'm here")
# # contour_plot(coords, dict)
# contour_3D(coords, dict, hold=False)

In [ ]:
import numpy as np


np.set_printoptions(suppress=True)  # Prevent numpy exponential notation on print, default False

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


from numpy.random import rand
def compute_velocity(v, g, P, p, params):
    (c1, c2) = params['ac']
    w = params['iw']
    r_p, r_g = rand(), rand()
    new_v = w*v + c1*r_p * (p - P) + c2*r_g * (g - P)
    return new_v


def selection(f_current, f_prev, maximize=False):
    if maximize:
        return np.where(f_current > f_prev)
    return np.where(f_current < f_prev)

def ring_selection(f_p, maximize=False):
    comparer = max if maximize else min
    indices = []
    n = len(f_p)
    for i in range(n - 1):
        neighbors = (f_p[i-1], f_p[i], f_p[i+1])
        idx = neighbors.index(comparer(neighbors))
        indices.append(idx + (i-1))

    neighbors_last = (f_p[n-2], f_p[n-1], f_p[0])
    idx_last = neighbors_last.index(comparer(neighbors_last))
    if idx_last != 2:
        indices.append(idx_last + n-2)
    else:
        indices.append(0)

    return indices

def star_selection(f_p, maximize=False):
    comparer = np.argmax if maximize else np.argmin
    return comparer(f_p, axis=0)

def select_best_neighbors(mode, f_p, maximize):
    assert(mode != 'star' or mode != 'ring')

    if mode == 'star':
        return star_selection(f_p, maximize)
    if mode == 'ring':
        return ring_selection(f_p, maximize)
    

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy.random import uniform
def optimize(params, plot=0, print_scr=False):
    """

    """

    # Initialize required parameters from dictionary
    num_inds = params['N']
    max_gen = params['g']
    seed = params['s']
    maximize = params['maximize']

    selection_mode = params['sm']

    f_dict = params['f']    # Dictionary of fitness function data
    num_params = f_dict['d']
    (lower_bound, upper_bound) = f_dict['D']
    real_valued = f_dict['real valued']

    # Plot search space
    plottable = plot and num_params == 2
    if plottable:
        data = get_plot_data(f_dict)
        fig, ax = plt.subplots()
        if plottable and plot == 2:
            ax = Axes3D(fig)
    
    # Initialize
    comparer = np.argmax if maximize else np.argmin
    np.random.seed(seed)
    epsilon = 10**-5
    P = initialize(num_inds, num_params, 
                   domain=(lower_bound, upper_bound), 
                   real_valued=real_valued)
    f_P = None
    p = P.copy()
    v = uniform(low=-abs(upper_bound-lower_bound), 
                high=abs(upper_bound-lower_bound), 
                size=(num_inds, num_params)).astype(np.float64)
    gen = 0
    num_f_func_calls = 0
    #
    while not pop_converge(P):
        gen += 1
        if max_gen_reached(gen, max_gen):
            break

        # Evaluate
        f_P = evaluate(P, f_dict['function'])
        f_p = evaluate(p, f_dict['function'])

        num_f_func_calls += len(f_P) * 2

        # Selection
        selected_indices = selection(f_P, f_p, maximize)
        p[selected_indices] = P[selected_indices]
        f_p[selected_indices] = f_P[selected_indices]

        g_indices = select_best_neighbors(selection_mode, f_p, maximize)
        g = p[g_indices]

        # Variate
        v = compute_velocity(v, g, P, p, params)
        P = (sigmoid(v) > 0.5).astype(np.int)
        #

        # Visualize / log result
        if print_scr and gen % 100 == 0:
            print('## Gen {}: {} (Fitness: {})'.format(gen, P[comparer(f_P)].reshape(1, -1), 
                                                       f_P[comparer(f_P)]))
        if plottable:
            ax.clear()
            if plot == 1:
                contour_plot(ax, data, f_dict, hold=True)
                ax_lim = (xlim, ylim) = ax.get_xlim(), ax.get_ylim()
                scatter_plot(ax_lim, ax, P, hold=True)
            else:
                contour_3D(ax, data, f_dict, hold=True)
                ax_lim = (xlim, ylim, zlim) = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
                scatter_3D(ax_lim, ax, P, f_P, hold=True)
            plt.pause(epsilon)
        #

    if plottable: 
        plt.show()     

    solution =  P[comparer(f_P)].reshape(1, -1).flatten()
    opt_sol_found = None

    optimize_goal = 'global maximum' if maximize else 'global minimum'
    if type(f_dict[optimize_goal]) != type(None):
        diffs = np.abs(f_dict[optimize_goal] - solution).sum(axis=1)
        opt_sol_found = len(np.where(diffs <= num_params*epsilon)[0]) != 0

    result = { 'solution' : solution, 
               'evaluate function calls' : num_f_func_calls, 
               'global optima found' : opt_sol_found }
    return result


def get_parameters(**params):
    mode = 'star' if params['mode'] == "" or params['mode'] not in ['star', 'ring'] else params['mode']
    default_params = { 'N' : params['N'],
                       's' : params['s'],
                       'iw' : 0.7298,
                       'ac' : (1.49618, 1.49618),
                       'g' : params['g'],
                       'sm' : mode,
                       'maximize' : params['maximize'],
                       'f' : params['f']}
    return default_params